<a href="https://colab.research.google.com/github/rpatel71/Design-Optimization/blob/main/Homework_4/HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Design Optimization: Homework-4: Problem-4

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import torch as t
from torch.autograd import Variable
from torch.autograd.functional import jacobian
from numpy.linalg import inv

In [11]:
fun = lambda x: (x[0] ** 2 + x[1] ** 2 + x[2] ** 2)   # Objective function
const1 = lambda x: ((x[0] ** 2) / 4) + ((x[1] ** 2) / 5) + (((x[2] ** 2) / 25) - 1)   # Constraint 1 from the given constraints
const2 = lambda x: (x[0] + x[1] - x[2])   # Constraint 2 from the given constaints

h = np.array([const1, const2])   # h is an array of constraints

x = Variable(t.tensor([1, 1., 1.]), requires_grad=True)

Here, we know that, we have n=3 and constraints m=2. 

So, we have Decision variables = n - m = 1, and State variables = m = 2.

So, Decision variables (d) = [x1] and state variables (s) = [x2, x3]

In [21]:

jac = t.zeros(3, 3)
jac[0] = jacobian(fun, x)
jac[1] = jacobian(const1, x)
jac[2] = jacobian(const2, x)

dfdd = jac[0,0]
dfds = jac[0,1:]
dhds = jac[1:,1:]
dhdd = jac[1:,0]
print(jac)
print(dfdd)
print(dfds)
print(dhds)
print(dhdd)

tensor([[ 2.0000,  2.0000,  2.0000],
        [ 0.5000,  0.4000,  0.0800],
        [ 1.0000,  1.0000, -1.0000]])
tensor(2.)
tensor([2., 2.])
tensor([[ 0.4000,  0.0800],
        [ 1.0000, -1.0000]])
tensor([0.5000, 1.0000])
